In [1]:
# make a comparison of just the different blood groups
# Single Var pred : Only Blood Type and no other features-
# Pick the 5 and 10 best features and 

In [2]:
import pandas as pd
import numpy as np
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve 

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.svm import SVC

from sklearn.model_selection import KFold, cross_val_score

import copy
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
unneeded_data = ['Place_of_isolation','Patient_In','Patient_out']
catigorical_data=['Blood_Group','CONTROL_blood_group','Risk_Factor']
labels = ['Severeity','Outcome']
droped_col = unneeded_data+catigorical_data+labels

NameError: name 'df' is not defined

In [ ]:
df = pd.read_csv("Covid-19_dataset.csv",header=1)


df['is_Male'] = pd.get_dummies(df['Gender'])['MALE']#one hot encoding gender so male = #1 and female = 0

blood_types = pd.get_dummies(df['Blood_Group']).columns
blood_group_columns = pd.get_dummies(df[['Blood_Group','CONTROL_blood_group']]).columns


df = pd.concat([df, pd.get_dummies(df[['Blood_Group','CONTROL_blood_group','Risk_Factor','Infection_type']])], axis=1)



df = df.drop(['Blood_Group','CONTROL_blood_group','Risk_Factor','Infection_type','Gender','Place_of_isolation','Patient_In','Patient_out'],axis=1)

columns = df.columns



# getting rid of blank values
print("before",df.shape)
for name in columns:
    df = df[df[name] != " "]
    df = df.dropna()
    
label_Severeity = df['Severeity'] #Severeity vs Outcome
label_Outcome = df['Outcome']
df = df.drop(['Severeity','Outcome'],axis=1)


df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))#strings to numeric
all_columns = df.columns
# df = df.reset_index()

print("after",df.shape)





In [ ]:
blood_type_labels =[]
control_blood_type_labels = []
for i in blood_group_columns:
    if("CONTROL_" in i):
        control_blood_type_labels.append(i)
    else:
        blood_type_labels.append(i)

In [ ]:
df_blood_data = copy.deepcopy(df[blood_type_labels])
df_CONTROL_blood_data = copy.deepcopy(df[control_blood_type_labels])

In [ ]:
df_blood_data

In [ ]:
df_CONTROL_blood_data

In [ ]:
df_blood_data[df_blood_data['Blood_Group_A+']== 1]

In [ ]:
def CrossValAndAUC(model,K=10): 
    plt.figure()
    %matplotlib inline
    
    for bt in blood_type_labels:
        Blood_Data = df_blood_data[df_blood_data[bt]== 1]
        y = label_Outcome[df_blood_data[bt]== 1]

        X_train, X_test, y_train, y_test = train_test_split(Blood_Data, y, test_size=0.25, random_state=1)
        
        if(len(y_train.unique()) == 1 or len(y_test.unique()) == 1):
            continue

        # Training ONLY on the training set to get AUC:
        tempModel = copy.deepcopy(model)
        model.fit(X_train, y_train)
        
        prob = model.predict_proba(X_test)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, prob[:,1], pos_label='Recovered')
        AUC = metrics.auc(fpr, tpr)
        print("AUC : ",AUC)
        plt.plot(fpr, tpr,label=f"With Blood AUC: {AUC}")


    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.figlegend()
    plt.show()


In [ ]:
print("Logistic Regression")
my_logreg = LogisticRegression(max_iter=10000)
CrossValAndAUC(my_logreg)

In [ ]:
print("Decision Tree")
my_DecisionTree = DecisionTreeClassifier(random_state=4)
CrossValAndAUC(my_DecisionTree)

In [ ]:
print("Random Forest Classifier")
my_RandomForest = RandomForestClassifier(n_estimators=29, bootstrap=True, random_state=4)
CrossValAndAUC(my_RandomForest)

In [ ]:
print("Ada Boost Classifier")
my_AdaBoost = AdaBoostClassifier(n_estimators=14, random_state=4)
CrossValAndAUC(my_AdaBoost)

In [ ]:
print("SVM")
my_svm = SVC(kernel='rbf', probability=True)
CrossValAndAUC(my_svm)

In [ ]:


# def CrossValAndAUC(model,K=10): 
#     plt.figure()
#     %matplotlib inline

#     for bt in blood_type_labels:
#         Blood_Data = df_blood_data[df_blood_data[bt]== 1]
#         y = label_Outcome[df_blood_data[bt]== 1]
        
        
#         if(len(y.unique()) == 1):
#             print(f"Only {y.unique()[0]} samples for the {bt}")
#             continue;


# #         crossVal_model = copy.deepcopy(model)
# #         #10 Fold cross validation
# #         kf = KFold(n_splits=K, shuffle=True)
# #         scores = cross_val_score(crossVal_model, Blood_Data, y, cv=kf)
# #         print(f" {bt} | {K} Forld cross validation average: {scores.mean()}")

#         model.fit(Blood_Data,y)




#         # Training ONLY on the training set to get AUC:
#         tempModel = copy.deepcopy(model)
#         model.fit(Blood_Data, y)
#         predict = model.predict(Blood_Data)
#         prob = model.predict_proba(Blood_Data)
#         fpr, tpr, thresholds = metrics.roc_curve(y, prob[:,1], pos_label='Recovered')
#         AUC = metrics.auc(fpr, tpr)
#         print("AUC : ",AUC)
#         plt.plot(fpr, tpr,label=f"{bt}: {AUC}")


#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('ROC Curve')
#     plt.figlegend()
#     plt.show()
